## Анализ тональности отзывов на фильмы: строим простые модели

В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

Мы будем использовать стандартный датасет из nltk, уже возникавший в одном из примеров в предыдущих курсах. Для того, чтобы импортировать необходимый модуль, напишите:

In [168]:
from nltk.corpus import movie_reviews

Чтобы получить id-шники негативных и позитивных отзывов:

In [169]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

Чтобы получить список негативных отзывов:

In [170]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]

Инструкция по выполнению

В некоторых пунктах нужно получить ответ - число или строку, которые будет нужно набирать в текстовых файлах и прикреплять в ответах на вопросы. Десятичные дроби записывайте через точку.

1. Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.
2. Подсчитайте количество отзывов в выборке.

In [183]:
import pandas as pd
# список положительных отзывов
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]
# сделаю табличку с отзывами
feats = pd.DataFrame(columns = ['idx', 'feat', 'class'])
# всего отзывов - сумма количества id-шников
print(len(posids) + len(negids))

2000


In [184]:
def add_data(df, data, cls):
    j = len(df)
    for i in range(len(data)):
        df.loc[j] = [data[i], list(movie_reviews.words(fileids=[data[i]])), cls]
        #df.loc[j] = [data[i], ' '.join(movie_reviews.words(fileids=[data[i]])), cls]
        j += 1
    return df

In [185]:
feats = add_data(feats, negids, 0)

In [187]:
feats = add_data(feats, posids, 1)

In [188]:
feats

,idx,feat,class
0,neg/cv000_29416.txt,"[plot, :, two, teen, couples, go, to, a, churc...",0
1,neg/cv001_19502.txt,"[the, happy, bastard, ', s, quick, movie, revi...",0
2,neg/cv002_17424.txt,"[it, is, movies, like, these, that, make, a, j...",0
3,neg/cv003_12683.txt,"["", quest, for, camelot, "", is, warner, bros, ...",0
4,neg/cv004_12641.txt,"[synopsis, :, a, mentally, unstable, man, unde...",0
...,...,...,...
1995,pos/cv995_21821.txt,"[wow, !, what, a, movie, ., it, ', s, everythi...",1
1996,pos/cv996_11592.txt,"[richard, gere, can, be, a, commanding, actor,...",1
1997,pos/cv997_5046.txt,"[glory, --, starring, matthew, broderick, ,, d...",1
1998,pos/cv998_14111.txt,"[steven, spielberg, ', s, second, epic, film, ...",1


In [44]:
def write_answer(numb_answ, *answ):
    file_name = 'answer_' + str(numb_answ) + '.txt'
    with open(file_name, 'w') as f:
        f.write(' '.join(answ))

In [45]:
write_answer(1, '2000')

3. Подсчитайте долю класса 1 в выборке.

In [46]:
dp = len(posids) / (len(posids) + len(negids))
dp

0.5

In [51]:
write_answer(2, str(dp))

4. Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [189]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [206]:
Vect = vectorizer.fit_transform(movie_reviews.words())
#Vect = vectorizer.fit_transform(feats['feat'])

In [209]:
print('Количество признаков : {}'.format(len(vectorizer.get_feature_names())))

Количество признаков : 39659


In [85]:
write_answer(3, '39659')

5. Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [226]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np

In [215]:
logreg = LogisticRegression(solver='liblinear')

In [216]:
pipe = Pipeline(steps=[('vectorizer', vectorizer), ('LR', logreg)])

In [235]:
X = [' '.join(x) for x in feats['feat']]
y = np.array(feats['class']).astype('int')

In [236]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75)

In [244]:
score = cross_val_score(pipe, X, y, cv=5)

In [245]:
score.mean()

0.8415000000000001

In [275]:
write_answer(4, '0.84')

6. Аналогично accuracy, оцените качество по ROC AUC.

In [278]:
roc_auc = cross_val_score(pipe, X, y, cv=5, scoring='roc_auc')
roc_auc.mean()

0.91605

In [279]:
write_answer(5, '0.916')

7. Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [248]:
New_LR = LogisticRegression(solver='liblinear')

In [250]:
X_t = vectorizer.fit_transform(X)

In [255]:
New_LR.fit(X_t, y)

LogisticRegression(solver='liblinear')

In [273]:
coeffs = New_LR.coef_[0]

In [274]:
[vectorizer.get_feature_names()[list(coeffs).index(i)] for i in sorted(coeffs)[:5]]

['bad', 'unfortunately', 'worst', 'waste', 'nothing']

In [277]:
write_answer(6, 'bad unfortunately')